In [63]:
import random
from datetime import datetime, timedelta
import networkx as nx
from pyvis.network import Network
from collections import deque
import community

### 예시 데이터 만들기

In [88]:
transactions = []
names = [
    '신소율', '홍수아', '홍지호', '신민지', '신도윤', '한민준', '안현우', '최하은', '권현준', '홍지은',
    '윤지원', '김태윤', '이주원', '박서준', '최예은', '강지호', '배수아', '정하늘', '오유진', '서민재',
    '장하준', '민서연', '김다은', '송지후', '이하린', '윤지우', '한지호', '최다은', '정민준', '서지우'
]

start_time = datetime(2024, 1, 1, 9, 0, 0)

for _ in range(50):
    sender = random.choice(names)
    receiver = random.choice(names)

    # 출금인과 송금인이 같으면 재선택
    while sender == receiver:
        receiver = random.choice(names)
    
    amount = random.randint(10000, 5000000)
    time_delta = timedelta(minutes=random.randint(0, 60*24*30))
    transaction_time = (start_time + time_delta).strftime('%m-%d %H:%M')

    transactions.append((sender, receiver, amount, transaction_time))

In [89]:
transactions

[('권현준', '안현우', 82528, '01-11 19:51'),
 ('김다은', '배수아', 197242, '01-08 02:47'),
 ('홍지은', '박서준', 4462689, '01-26 10:35'),
 ('한지호', '민서연', 3830616, '01-30 14:20'),
 ('홍지호', '안현우', 781259, '01-08 09:28'),
 ('서지우', '김태윤', 1338553, '01-22 05:18'),
 ('이주원', '윤지우', 3123589, '01-15 10:45'),
 ('김다은', '신도윤', 319516, '01-19 22:29'),
 ('윤지우', '홍수아', 2929597, '01-22 22:27'),
 ('서민재', '홍수아', 4303194, '01-11 23:48'),
 ('안현우', '이주원', 1893402, '01-08 12:06'),
 ('윤지원', '홍수아', 4871681, '01-12 07:22'),
 ('홍지호', '최예은', 585547, '01-29 01:49'),
 ('신도윤', '장하준', 4333022, '01-18 02:18'),
 ('권현준', '오유진', 3318549, '01-21 08:18'),
 ('홍지은', '김다은', 2271280, '01-19 19:32'),
 ('최다은', '최예은', 554961, '01-06 02:09'),
 ('최다은', '최하은', 2759632, '01-25 01:09'),
 ('박서준', '윤지원', 3631210, '01-03 18:34'),
 ('신민지', '배수아', 1736226, '01-20 00:08'),
 ('안현우', '신소율', 1909270, '01-29 08:59'),
 ('박서준', '신소율', 1940992, '01-30 04:13'),
 ('이하린', '홍수아', 777622, '01-09 02:37'),
 ('강지호', '홍수아', 3555027, '01-02 00:04'),
 ('홍수아', '오유진', 2141805,

### 전체 그래프 그리기

In [90]:
# 전체 그래프 그리기
net = Network(height='800px', width='100%', directed=True)

for sender, receiver, amount, time in transactions:
    net.add_node(sender, label=sender)
    net.add_node(receiver, label=receiver)
    net.add_edge(sender, receiver, title=f"{amount}원\n{time}", value=amount)

net.save_graph('full_graph2.html')

In [87]:
# 부분 그래프 그리기
G = nx.DiGraph()
for sender, receiver, amount, time in transactions:
    G.add_edge(sender, receiver, amount=amount, time=time)

# BFS (너비 우선 탐색)
def find_edges(G, start_node, max_depth):
    visited = set()
    visited.add(start_node) # 시작 노드 방문 추가
    queue = deque()
    queue.append((start_node, 0))
    result_edges = set()

    while queue:
        node, depth = queue.popleft()
        if depth >= max_depth:
            continue
        # 출금 (내가 돈을 보낸 사람)
        for neighbor in G.successors(node): 
            result_edges.add((node, neighbor))
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, depth+1))
        # 입금 (나에게 돈을 보낸 사람)
        for neighbor in G.predecessors(node):
            result_edges.add((neighbor, node))
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, depth+1))
    return list(result_edges)

start_node = '김태윤'
max_depth = 2
sub_edges = find_edges(G, start_node, max_depth)

# pyvis 시각화
net = Network(height='800px', width='100%', directed=True)
added_nodes = set()
for sender, receiver in sub_edges:
    for node in (sender, receiver):
        if node not in added_nodes:
            net.add_node(node, lable=node)
            added_nodes.add(node)
    edge_data = G.get_edge_data(sender, receiver)
    if edge_data: # edge_data가 있으면
        net.add_edge(sender, receiver, title=f"{edge_data['amount']}원\n{edge_data['time']}", value=edge_data['amount'])
    else:
        # edge_data가 없을 수도 있으므로 예외 처리
        net.add_edge(sender, receiver)

# 중심 노드 강조
net.get_node(start_node)['color'] = 'red'

# 저장
output_path = 'graph.html'
net.save_graph(output_path)
output_path

'graph.html'

### 커뮤니티 디텍션

In [91]:
# Louvain, Leidan같은 커뮤니티 디텍션 알고리즘들은
# 방향성 없는 그래프를 기본으로 가정
G = nx.DiGraph()
for sender, receiver, amount, time in transactions:
    G.add_edge(sender, receiver, amount=amount, time=time)

G_undirected = G.to_undirected()

# 커뮤니티 검출
partition = community.best_partition(G_undirected)

# 결과
partition

{'권현준': 2,
 '안현우': 3,
 '김다은': 2,
 '배수아': 2,
 '홍지은': 4,
 '박서준': 4,
 '한지호': 4,
 '민서연': 4,
 '홍지호': 1,
 '서지우': 1,
 '김태윤': 1,
 '이주원': 3,
 '윤지우': 3,
 '신도윤': 2,
 '홍수아': 0,
 '서민재': 0,
 '윤지원': 4,
 '최예은': 2,
 '장하준': 2,
 '오유진': 2,
 '최다은': 2,
 '최하은': 3,
 '신민지': 2,
 '신소율': 3,
 '이하린': 4,
 '강지호': 0,
 '정민준': 0,
 '정하늘': 4,
 '송지후': 0,
 '한민준': 3}

### 시각화에 커뮤니티 반영

In [ ]:
color_list = ['red', 'blue', 'green', 'purple', 'pink', 'brown', 'cyan']

net = Network(height='800px', width='100%', directed=True)

for node in G.nodes():
    comm_id = partition.get(node, 0) # 커뮤니티 id 추출
    color = color_list[comm_id % len(color_list)]
    net.add_node(node, label=node, color=color)

for sender, receiver in G.edges():
    edge_data = G.get_edge_data(sender, receiver)
    net.add_edge(sender, receiver, title=f"{edge_data['amount']}원\n{edge_data['time']}", value=edge_data['amount'])

net.save_graph('community2.html')

### 특정 커뮤니티만 시각화

In [99]:
comm_number = 0
selected_community = []

color_list = ['red', 'blue', 'green', 'purple', 'pink', 'brown', 'cyan']

for node, comm_id in partition.items():
    if comm_id == comm_number:
        selected_community.append(node)

net = Network(height='800px', width='100%', directed=True)

# 노드 먼저 추가
for node in selected_community:
    net.add_node(node, label=node, color=color_list[comm_number % len(color_list)])

for sender, receiver in G.edges():
    if (sender in selected_community) and (receiver in selected_community):
        edge_data = G.get_edge_data(sender, receiver)
        net.add_edge(sender, receiver, title=f"{edge_data['amount']}원\n{edge_data['time']}", value=edge_data['amount'])

net.save_graph('selected_community.html')
